In [1]:
import datasets
import torch
from transformers import BeitForImageClassification, BeitFeatureExtractor, BeitImageProcessor
import numpy as np

base_model_name = "microsoft/beit-base-patch16-224-pt22k"

processor = BeitImageProcessor.from_pretrained(base_model_name)

def crop_image(img):
    width, height = img.size
    target_size = min(width, height)
    left = max(0, (width - target_size ) // 2)
    right = left + target_size
    top = max(0, (height - target_size ) // 2)
    bottom = top + target_size
    return img.crop((left, top, right, bottom)).convert('RGB')

def process_example(example):
    img = crop_image(example['image'])
    inputs = processor(img, return_tensors='pt')
    inputs['labels'] = example['label']
    return inputs

def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = processor([crop_image(x) for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = example_batch['label']
    return inputs

ds = datasets.load_from_disk("../../data/diffusion_and_real/")

prepared_ds = ds.with_transform(transform)


def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

metric = datasets.load_metric("f1")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

labels = ["negative","positive"]

model = BeitForImageClassification.from_pretrained(
    base_model_name,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
).to("cuda")

C:\Users\Tim\AppData\Local\Temp/ipykernel_11564/3893446471.py:44: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("f1")
Some weights of the model checkpoint at microsoft/beit-base-patch16-224-pt22k were not used when initializing BeitForImageClassification: ['layernorm.bias', 'lm_head.bias', 'lm_head.weight', 'layernorm.weight']
- This IS expected if you are initializing BeitForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BeitForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

In [2]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./BEiT-diff-detect",
  per_device_train_batch_size=8, # consider 16 or higher
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=6000,
  eval_steps=12,
  logging_steps=1125,
  learning_rate=2e-4,
  save_total_limit=3,
  logging_first_step=True,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["vali"],
    tokenizer=processor,
)

In [3]:
# start the training
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

c:\Users\Tim\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 2/9000 [00:09<10:00:28,  4.00s/it]

{'loss': 0.7854, 'learning_rate': 0.0002, 'epoch': 0.0}


                                                   
  0%|          | 13/9000 [00:19<6:39:07,  2.66s/it]

{'eval_loss': 0.8682252168655396, 'eval_f1': 0.0, 'eval_runtime': 8.1714, 'eval_samples_per_second': 122.379, 'eval_steps_per_second': 15.297, 'epoch': 0.01}


                                                   
  0%|          | 25/9000 [00:28<6:06:16,  2.45s/it]

...

{'loss': 0.1737, 'learning_rate': 1.5555555555555556e-07, 'epoch': 4.0}


                                                     
100%|██████████| 9000/9000 [3:13:07<00:00,  3.70it/s]

{'eval_loss': 0.27891668677330017, 'eval_f1': 0.9268795056642637, 'eval_runtime': 11.2633, 'eval_samples_per_second': 88.784, 'eval_steps_per_second': 11.098, 'epoch': 4.0}


100%|██████████| 9000/9000 [3:13:07<00:00,  1.29s/it]


{'train_runtime': 11587.4068, 'train_samples_per_second': 6.214, 'train_steps_per_second': 0.777, 'train_loss': 0.2982212431165907, 'epoch': 4.0}
***** train metrics *****
  epoch                    =        4.0
  train_loss               =     0.2982
  train_runtime            = 3:13:07.40
  train_samples_per_second =      6.214
  train_steps_per_second   =      0.777


In [4]:
# eval
metrics = trainer.evaluate(prepared_ds['test'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

100%|██████████| 125/125 [00:11<00:00, 11.24it/s]

***** eval metrics *****
  epoch                   =        4.0
  eval_f1                 =     0.9256
  eval_loss               =     0.2209
  eval_runtime            = 0:00:11.19
  eval_samples_per_second =     89.296
  eval_steps_per_second   =     11.162
